In [2]:
import json

#Loading Data
with open("intents.json") as file:
	data = json.load(file)

In [3]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [4]:
docs_x = []
docs_y = []
labels= []

In [5]:
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
import string
#from nltk.stem.lancaster import LancasterStemmer

#stemming good?
#stemmer = LancasterStemmer()
rt = RegexpTokenizer(r'[^\W_]+|[^\W_\s]+')
stop = set(stopwords.words('english') + list(string.punctuation))
#Looping through our data
for intent in data['intents']:
  words = []
  for pattern in intent['patterns']:
    pattern = pattern.lower()
    #print(pattern)
    #Creating a list of words
    ###wrds = rt.tokenize(pattern)
    wrds = [i for i in rt.tokenize(pattern) if i not in stop]
    words.append(wrds)
  docs_x.append(words)
  docs_y.append(intent['tag'])

In [6]:
print(docs_x)
print(docs_y)
print(len(docs_y))

[[['skin', 'rash'], ['dischromic', 'patches'], ['itching'], ['nodal', 'skin', 'eruptions']], [['chills'], ['shivering'], ['continuous', 'sneezing']], [['vomiting'], ['chest', 'pain'], ['acidity'], ['ulcers', 'tongue'], ['stomach', 'pain'], ['cough']], [['vomiting'], ['nausea'], ['yellowish', 'skin'], ['abdominal', 'pain'], ['loss', 'appetite'], ['yellowing', 'eyes'], ['itching']], [['burning', 'micturition'], ['skin', 'rash'], ['spotting', 'urination'], ['stomach', 'pain'], ['itching']], [['vomiting'], ['indigestion'], ['passage', 'gases'], ['loss', 'appetite'], ['abdominal', 'pain'], ['internal', 'itching']], [['extra', 'marital', 'contacts'], ['patches', 'throat'], ['high', 'fever'], ['muscle', 'wasting']], [['lethargy'], ['increased', 'appetite'], ['fatigue'], ['obesity'], ['irregular', 'sugar', 'level'], ['restlessness'], ['weight', 'loss'], ['blurred', 'distorted', 'vision'], ['excessive', 'hunger'], ['polyuria']], [['diarrhoea'], ['vomiting'], ['dehydration'], ['sunken', 'eyes']]

In [7]:
import progressbar
import urllib.request

pbar = None
def show_progress(block_num, block_size, total_size):
    global pbar
    if pbar is None:
        pbar = progressbar.ProgressBar(maxval=total_size)
        pbar.start()

    downloaded = block_num * block_size
    if downloaded < total_size:
        pbar.update(downloaded)
    else:
        pbar.finish()
        pbar = None


#urllib.request.urlretrieve(model_url, model_file, show_progress)
urllib.request.urlretrieve('https://nlp.stanford.edu/data/glove.6B.zip','glove.6B.zip', show_progress)

100% (862182613 of 862182613) |##########| Elapsed Time: 0:02:40 Time:  0:02:40


('glove.6B.zip', <http.client.HTTPMessage at 0x7ffa6b417e50>)

In [8]:
!unzip "/content/glove.6B.zip" -d "/content/"

Archive:  /content/glove.6B.zip
  inflating: /content/glove.6B.50d.txt  
  inflating: /content/glove.6B.100d.txt  
  inflating: /content/glove.6B.200d.txt  
  inflating: /content/glove.6B.300d.txt  


In [9]:
import numpy as np

embeddings_index = dict()
f = open('./glove.6B.300d.txt')

for line in f:
    word, coefs = line.split(maxsplit=1)
    coefs = np.fromstring(coefs, "f", sep=" ")
    embeddings_index[word] = coefs

print("Found %s word vectors." % len(embeddings_index))

Found 400000 word vectors.


In [10]:
#testing embeddings

embedding_vector = embeddings_index.get("ulcer")
print(embedding_vector.shape)
arr = np.zeros(300)

(300,)


In [11]:
diseases = 0
word_count = 0
max_symptoms_count = 0
for symptoms_of_1_disease in docs_x:
  diseases += 1
  max_symptoms_count = max(max_symptoms_count, len(symptoms_of_1_disease))
  for individual_symptom in symptoms_of_1_disease:
    for word in individual_symptom:
      word_count+=1
print(diseases)
print(max_symptoms_count)
print(word_count)

41
17
543


In [12]:
len(docs_x)

41

In [13]:
import keras
import random

class DataGen(keras.utils.all_utils.Sequence):
    def __init__(self, docs_x, output, batch_size=8):
        self.docs_x = docs_x
        self.output = output
        self.batch_size = batch_size
        self.on_epoch_end()
        self.hits=0
        self.misses=0
    
    def __getitem__(self, index):
        if((index+1)*self.batch_size > len(self.docs_x)):
            self.batch_size = len(self.docs_x) - index*self.batch_size
        
        inp_batch = self.docs_x[index*self.batch_size : (index+1)*self.batch_size]
        #print()
        inp_batch_embeddings = [] #np.zeros(shape = (self.batch_size, max_symptoms_count, 300))
        for symptoms_of_1_disease in inp_batch:
          random.shuffle(symptoms_of_1_disease)
          symptoms_embedding = np.zeros((len(symptoms_of_1_disease), 300))
          j = 0
          for individual_symptom in symptoms_of_1_disease:
            individual_symptom_embedding = np.zeros(300)
            for word in individual_symptom:
              word_embedding = embeddings_index.get(word)
              if word_embedding is not None:
                self.hits += 1
                individual_symptom_embedding += word_embedding
              else:
                self.misses += 1
            symptoms_embedding[j] = individual_symptom_embedding #new code add padding using DataGen later
            j = j + 1
          symptoms_embedding = np.pad(symptoms_embedding, [((max_symptoms_count - symptoms_embedding.shape[0]), 0), (0, 0)], mode='constant', constant_values=0)
          inp_batch_embeddings.append(symptoms_embedding)
        
        inp_batch_embeddings = np.array(inp_batch_embeddings)

        out_batch = self.output[index*self.batch_size : (index+1)*self.batch_size]
     
        return inp_batch_embeddings, out_batch
    
    def on_epoch_end(self):
        pass
    
    def __len__(self):
        return int(np.ceil(len(self.docs_x)/float(self.batch_size)))

In [14]:
#label encoding y axis

from sklearn import preprocessing

label_encoder = preprocessing.LabelEncoder()  
y =  label_encoder.fit_transform(docs_y)
y= np.asarray(y)
print(y)

[15  4 16  9 14 33  1 12 17  6 23 30  7 32 28 29  8 11 37 40 19 20 21 22
  3 36 10 34 13 18 39 26 24 25 31  5  0  2 38 35 27]


In [15]:
# from keras.models import *
# from keras.layers import *

# def define_model():
#     input1 = Input(shape=(max_symptoms_count,300)) 
#     lstm1 = Bidirectional(LSTM(units=128))(input1)
#     dnn_hidden_layer1 = Dense(64, activation='relu')(lstm1) 
#     dnn_output = Dense(41, activation='softmax')(dnn_hidden_layer1)
#     model = Model(inputs=[input1],outputs=[dnn_output])
#     # compile the model
#     model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
#     model.summary()
#     return model
# model = define_model()

In [16]:
#Add attention layer to the deep learning network
from pandas import read_csv
import numpy as np
from keras import Model
from keras.layers import Layer
import keras.backend as K
from keras.layers import Input, Dense, SimpleRNN,LSTM
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.metrics import mean_squared_error

class attention(Layer):
    def __init__(self,**kwargs):
        super(attention,self).__init__(**kwargs)
 
    def build(self,input_shape):
        self.W=self.add_weight(name='attention_weight', shape=(input_shape[-1],1), 
                               initializer='random_normal', trainable=True)
        self.b=self.add_weight(name='attention_bias', shape=(input_shape[1],1), 
                               initializer='zeros', trainable=True)        
        super(attention, self).build(input_shape)
 
    def call(self,x):
        # Alignment scores. Pass them through tanh function
        e = K.tanh(K.dot(x,self.W)+self.b)
        # Remove dimension of size 1
        e = K.squeeze(e, axis=-1)   
        # Compute the weights
        alpha = K.softmax(e)
        # Reshape to tensorFlow format
        alpha = K.expand_dims(alpha, axis=-1)
        # Compute the context vector
        context = x * alpha
        context = K.sum(context, axis=1)
        return context
def create_RNN_with_attention(hidden_units, dense_units, input_shape, activation):
    x=Input(shape=input_shape)
    RNN_layer = SimpleRNN(hidden_units, return_sequences=True, activation=activation)(x)
    attention_layer = attention()(RNN_layer)
    outputs=Dense(dense_units, trainable=True, activation=activation)(attention_layer)
    model=Model(x,outputs)
    model.compile(loss='sparse_categorical_crossentropy', optimizer='adam',metrics=['accuracy'])    
    return model    
def create_LSTM_with_attention(hidden_units, dense_units, input_shape, activation):
    x=Input(shape=input_shape)
    LSTM_layer = LSTM(hidden_units, return_sequences=True, activation='relu')(x)
    attention_layer = attention()(LSTM_layer)
    outputs=Dense(dense_units, trainable=True, activation=activation)(attention_layer)
    model=Model(x,outputs)
    model.compile(loss='sparse_categorical_crossentropy', optimizer='adam',metrics=['accuracy'])    
    return model    
 
model = create_LSTM_with_attention(hidden_units=256, dense_units=41, input_shape=(max_symptoms_count,300), activation='softmax')
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 17, 300)]         0         
                                                                 
 lstm (LSTM)                 (None, 17, 256)           570368    
                                                                 
 attention (attention)       (None, 256)               273       
                                                                 
 dense (Dense)               (None, 41)                10537     
                                                                 
Total params: 581,178
Trainable params: 581,178
Non-trainable params: 0
_________________________________________________________________


In [17]:
batch_size = 8
train_gen = DataGen(
    docs_x,
    y,
    batch_size,
    )

train_steps = len(docs_x)//batch_size

history = model.fit(
    train_gen,
    steps_per_epoch=train_steps,
    epochs=200,
    verbose=1)

Epoch 1/200
5/5 [==============================] - 2s 44ms/step - loss: 3.7369 - accuracy: 0.0303
Epoch 2/200
5/5 [==============================] - 0s 28ms/step - loss: 3.4742 - accuracy: 0.4000
Epoch 3/200
5/5 [==============================] - 0s 28ms/step - loss: 3.5476 - accuracy: 0.0000e+00
Epoch 4/200
5/5 [==============================] - 0s 28ms/step - loss: 3.4739 - accuracy: 0.0000e+00
Epoch 5/200
5/5 [==============================] - 0s 29ms/step - loss: 3.8327 - accuracy: 0.0000e+00
Epoch 6/200
5/5 [==============================] - 0s 27ms/step - loss: 3.6234 - accuracy: 0.0000e+00
Epoch 7/200
5/5 [==============================] - 0s 28ms/step - loss: 3.8055 - accuracy: 0.0000e+00
Epoch 8/200
5/5 [==============================] - 0s 25ms/step - loss: 3.6131 - accuracy: 0.0000e+00
Epoch 9/200
5/5 [==============================] - 0s 29ms/step - loss: 3.5116 - accuracy: 0.4000
Epoch 10/200
5/5 [==============================] - 0s 29ms/step - loss: 3.2445 - accuracy: 0.

In [18]:
def predict_from_list(symptoms_of_1_disease):
  hits = 0
  misses = 0
  symptoms_embedding = np.zeros((max_symptoms_count-len(symptoms_of_1_disease), 300))
  for individual_symptom in symptoms_of_1_disease:
    individual_symptom_embedding = np.zeros(300)
    words = [i for i in rt.tokenize(individual_symptom) if i not in stop]
    print(words)
    for word in words:
      word_embedding = embeddings_index.get(word)
      if word_embedding is not None:
        hits += 1
        individual_symptom_embedding += word_embedding
      else:
        misses += 1
    symptoms_embedding = np.append(symptoms_embedding, [individual_symptom_embedding], axis = 0)
  #print(symptoms_embedding.shape)
  print(f"Of total words in symptoms: hits = {hits}, misses = {misses}, ratio = {hits/(hits+misses)}")
  test_data = np.array([symptoms_embedding])
  print(test_data.shape)
  pred = model.predict(test_data)
  pos = np.argmax(pred, axis=1)
  return label_encoder.inverse_transform([pos[0]])[0]

In [25]:
#lis = ["indigestion", "stiff_neck", "irritability","headache", "acidity"]

lis = [x for x in input("Enter symptoms separated by space: ").split(",")]
print(lis)
predicted_disease = predict_from_list(lis)
print(f"\n Predicted disease: {predicted_disease}")

Enter symptoms separated by space: muciod sputum, cough, family history, fatigue
['muciod sputum', ' cough', ' family history', ' fatigue']
['muciod', 'sputum']
['cough']
['family', 'history']
['fatigue']
Of total words in symptoms: hits = 5, misses = 1, ratio = 0.8333333333333334
(1, 17, 300)

 Predicted disease: Bronchial Asthma


In [26]:
def predict_from_sentence(symptoms):
  print(symptoms)
  hits = 0
  misses = 0
  symptoms_embedding = np.zeros((max_symptoms_count, 300))
  words = [i for i in rt.tokenize(symptoms) if i not in stop]
  print(words)
  for word in words:
    if hits == 17:
      break
    word_embedding = embeddings_index.get(word)
    if word_embedding is not None:
      hits += 1
      symptoms_embedding[max_symptoms_count-hits] += word_embedding
    else:
      word_embedding = np.zeros(300)
      misses += 1
    
  print(symptoms_embedding.shape)
  print(f"Of total words in symptoms: hits = {hits}, misses = {misses}, ratio = {hits/(hits+misses)}")
  test_data = np.array([symptoms_embedding])
  print(test_data.shape)
  pred = model.predict(test_data)
  pos = np.argmax(pred, axis=1)
  return label_encoder.inverse_transform([pos[0]])[0]

In [27]:
sentence = "i have skin rash dischromic patches itching and nodal skin eruptions"
predicted_disease = predict_from_sentence(sentence)
print(f"\n Predicted disease: {predicted_disease}")

i have skin rash dischromic patches itching and nodal skin eruptions
['skin', 'rash', 'dischromic', 'patches', 'itching', 'nodal', 'skin', 'eruptions']
(17, 300)
Of total words in symptoms: hits = 7, misses = 1, ratio = 0.875
(1, 17, 300)

 Predicted disease: Fungal infection


**Saving models**

In [57]:
import pickle 

filehandler = open("./chatbot_v1", 'wb') 
pickle.dump(model, filehandler)
filehandler.close()

INFO:tensorflow:Assets written to: ram://12e1fb93-2b0b-4576-91e8-33cde7af25ab/assets


INFO:tensorflow:Assets written to: ram://12e1fb93-2b0b-4576-91e8-33cde7af25ab/assets


In [60]:
filehandler = open("./chatbot_v1", 'rb') 
mo = pickle.load(filehandler)
filehandler.close()
print(mo)